# Data Science 101 - Part 2 of 2
## [Classifier Models](https://scikit-learn.org/stable/modules/tree.html#classification)

We'll use [scikit-learn](https://scikit-learn.org/stable/) to create a model that classifies some data (just created randomly) into 3 classes. This will be a good mockup of our cGMP datasets for the biosciences.

A classifier model separates 2 or more classes. For example, the blue and red lines below separate the filled circles from the empty circles. Each line would be a unique classifier (some are better than others as you can see).

![classifier image](https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Svm_separating_hyperplanes.png/1024px-Svm_separating_hyperplanes.png)

## Training a machine learning model to predict the class of a sample

We'll use [scikit-learn](https://scikit-learn.org) (aka sklearn) and [Dask](https://www.dask.org/) to build and train our classifier model.

### Scikit-learn - The Data Scientist's pocket knife

There are essentially 4 steps to building a model with scikit-learn:
1. Build the dataloader & get the train/test/validation splits
2. Construct the model
3. Train the model on the train split
4. Evaluate the model predictions on the test split

### Dask - Scale our data loader to a planetary level!

Here we'll use Dask as the dataloader. Recall from the first notebook (*python_final_1_of_2_scikit_learn.ipynb*) that a data loader is a generator that loads data only when it is needed. This allows us to use datasets that are far too large to fit into the computer's memory. It can also allow us to use streaming data that comes in continuously from a source (*e.g.* instruments in the lab that are constantly measuring things).

In the first notebook we created a dataloader from scratch. However, typically you would use something like Dask to do this because we don't want to re-invent the wheel. (Note, there are other Python libraries that do out-of-memory and streaming/incremental modeling).


### Validation versus Test Data Splits

There are 2 datasets that are typically "held back" from training: the test dataset and the validation dataset.

"Held back" means that this data is never seen by the model during training. Before we begin training we split the data (randomly) into train, test, and validation shards. Only the training shard is used for training. This is the only way to provide an unbiased estimate of the model's performance. Remember, we are fitting the model to the training dataset so it stands to reason that the model will perform very well on the training dataset. 

The model is never trained on the validation nor test shards. The validation dataset is used to give an unbiased estimate of the performance of the final tuned model when comparing or selecting between final models. The test dataset is used at the very end (once you select the best model) to give an unbiased estimate of how your best model will perform with new data.

## Let's create a new classifier dataset and save it to a CSV file

We'll use the [Iris](https://archive.ics.uci.edu/ml/datasets/iris) dataset which is a classic classifier dataset to predict the type of flower from its petal measurements. The dataset fits into memory, but we're going to pretend that it doesn't and load it with Dask. This will exactly match the realworld case when we have GB to TB sized CSV files.

In [1]:
import pandas as pd

In [2]:
csv_filename = "iris_classification_dataset.csv"
target_label = "Flower class"

In [3]:
from sklearn import datasets

data = datasets.load_iris()
df = pd.DataFrame(data=data["data"], columns=data["feature_names"])
df[target_label] = data["target"]
df.to_csv(csv_filename, sep=",", index=False)

## Now we have our CSV. Next step is using Dask to load it.

Note we're using Dask to replace the standard Pandas library. Think of Dask as Pandas on steroids.

In [4]:
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split

## Construct the model code


In [5]:
ddf = dd.read_csv(csv_filename, blocksize="16MB")

In [6]:
ddf

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Flower class
npartitions=1,,,,,
,float64,float64,float64,float64,int64
,...,...,...,...,...


In [7]:
ddf.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Flower class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [8]:
y = ddf[target_label]
X = ddf[ddf.columns.difference([target_label])]

X_train, X_remain, y_train, y_remain = train_test_split(
    X, y, test_size=0.3, shuffle=True, random_state=816
)

X_validate, X_test, y_validate, y_test = train_test_split(
    X_remain, y_remain, test_size=0.5, shuffle=True, random_state=816
)

In [9]:
y_train.head()

5      0
89     1
140    2
127    2
22     0
Name: Flower class, dtype: int64

In [10]:
X_train.head()

,petal length (cm),petal width (cm),sepal length (cm),sepal width (cm)
5,1.7,0.4,5.4,3.9
89,4.0,1.3,5.5,2.5
140,5.6,2.4,6.7,3.1
127,4.9,1.8,6.1,3.0
22,1.0,0.2,4.6,3.6


In [11]:
from tqdm.notebook import tqdm  # This gives us the nice progress bar!

## Time for the classifier model!

We're using the stochastic gradient descent classifier from scikit-learn. Note not all scikit-learn models will work with Dask. You'll need to find scikit-learn models that support the `partial_fit()` function. This function allows us to train one batch at a time.

In [12]:
from sklearn.linear_model import SGDClassifier
from dask_ml.wrappers import Incremental

In [13]:
classifier_model = SGDClassifier(loss="log_loss", verbose=True)  # Log loss is for classification models (use MSE for regression models)

## Normal scikit-learn with Dask

In the usual case, we can just use the `.fit()` method to train our model on our training data.

Just a single line!

In [14]:
classifier_model.fit(X_train, y_train)

-- Epoch 1
Norm: 119.82, NNZs: 4, Bias: 14.743384, T: 116, Avg. loss: 13.073869
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 108.53, NNZs: 4, Bias: 14.741709, T: 232, Avg. loss: 0.000002
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 99.19, NNZs: 4, Bias: 14.717165, T: 348, Avg. loss: 0.000027
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 91.33, NNZs: 4, Bias: 14.690104, T: 464, Avg. loss: 0.000032
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 84.65, NNZs: 4, Bias: 14.564809, T: 580, Avg. loss: 0.000171
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 78.86, NNZs: 4, Bias: 14.563823, T: 696, Avg. loss: 0.000001
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 73.81, NNZs: 4, Bias: 14.554370, T: 812, Avg. loss: 0.000015
Total training time: 0.00 seconds.
Convergence after 7 epochs took 0.00 seconds
-- Epoch 1
Norm: 117.49, NNZs: 4, Bias: 11.896649, T: 116, Avg. loss: 125.136553
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 117.32, NNZs: 4, Bias: 27.22

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished


SGDClassifier(loss='log_loss', verbose=True)

## Model performance

Let's see how accurate our model is.

In [15]:
print(f"The final model has an accuracy of {100*classifier_model.score(X_test, y_test):.1f}%")

The final model has an accuracy of 73.9%


In [16]:
df_scikit_learn = pd.DataFrame()

df_scikit_learn["Truth"] = y_test
df_scikit_learn["Prediction"] =  classifier_model.predict(X_test)

In [17]:
def map_values(x:int):
    """Map the class value to the class name
    
    Args:
        x(int): Class value
    """
    if x == 0:
        return "Iris Setosa"
    elif x == 1:
        return "Iris Virginica"
    elif x == 2:
        return "Iris Versicolour"
    else:
        return "Error - Class not known"

In [18]:
df_scikit_learn["Truth"] = df_scikit_learn["Truth"].map(map_values)
df_scikit_learn["Prediction"] = df_scikit_learn["Prediction"].map(map_values)

df_scikit_learn["Match?"] = (df_scikit_learn["Truth"] == df_scikit_learn["Prediction"])

df_scikit_learn

,Truth,Prediction,Match?
0,Iris Setosa,Iris Setosa,True
1,Iris Versicolour,Iris Virginica,False
2,Iris Setosa,Iris Setosa,True
3,Iris Setosa,Iris Setosa,True
4,Iris Versicolour,Iris Virginica,False
5,Iris Setosa,Iris Setosa,True
6,Iris Virginica,Iris Virginica,True
7,Iris Setosa,Iris Setosa,True
8,Iris Setosa,Iris Setosa,True
9,Iris Setosa,Iris Setosa,True


In [19]:
classifier_model.coef_

array([[ -53.75773825,  -28.05830848,   14.10604713,   39.63958043],
       [  16.09809505,  -81.4647437 ,   37.60991907,  -97.86839111],
       [ 151.5509483 ,  159.44470132, -137.07103959,  -89.24639181]])

## Incremental/batch/stream learning

So far everything has looked like our usual scikit-learn training. Here's where we tweak it a bit to allow for incremental learning. This might also be called batch learning or stream learning or online learning. It can handle datasets that are read in a batch at a time. This is wonderful when you are using really big datasets and datasets that are streaming from a source (like an IOT device).

In [20]:
classifier_model2 = SGDClassifier(loss="log_loss", verbose=True)  # Log loss is for classification models (use MSE for regression models)

In [21]:
classifier_increment = Incremental(classifier_model2, scoring="accuracy") 

In [22]:
NUMBER_OF_EPOCHS = 1000  # An epoch is just one pass through the dataset. We train for multiple epochs when we do SGD

In [23]:
classes = y_train.unique().compute()   # Note we need the .compute() at the end when using Dask (this is different than Pandas)
print(f"We have the following classes in our dataset:\n{classes}")

We have the following classes in our dataset:
0    0
1    1
2    2
Name: Flower class, dtype: int64


In [24]:
for _ in (pbar := tqdm(range(NUMBER_OF_EPOCHS))):
    
    # We're training for one batch at a time
    classifier_increment.partial_fit(X_train, y_train, classes=classes)
    
    pbar.set_description(f"Accuracy = {classifier_increment.score(X_train, y_train):.3f}")

  0%|          | 0/1000 [00:00<?, ?it/s]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished


-- Epoch 1
Norm: 42.79, NNZs: 4, Bias: 4.046831, T: 116, Avg. loss: 5.402490
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 102.75, NNZs: 4, Bias: 5.206546, T: 116, Avg. loss: 141.250835
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 217.42, NNZs: 4, Bias: -43.252076, T: 116, Avg. loss: 104.389202
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 38.76, NNZs: 4, Bias: 4.046830, T: 116, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 119.45, NNZs: 4, Bias: 14.205848, T: 116, Avg. loss: 122.783484
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 278.99, NNZs: 4, Bias: -54.678982, T: 116, Avg. loss: 55.587975
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 35.42, NNZs: 4, Bias: 4.046824, T: 116, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 179.91, NNZs: 4, Bias: 10.524950, T: 116, Avg. loss: 104.591569
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 296.94, NNZs: 4, Bias: -74.881147, T: 116, Avg. loss: 37.241093
Tot

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


-- Epoch 1
Norm: 26.35, NNZs: 4, Bias: 4.046212, T: 116, Avg. loss: 0.000001
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 165.15, NNZs: 4, Bias: 41.918154, T: 116, Avg. loss: 81.491716
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 340.94, NNZs: 4, Bias: -109.646029, T: 116, Avg. loss: 14.376026
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 24.76, NNZs: 4, Bias: 4.045319, T: 116, Avg. loss: 0.000001
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 165.38, NNZs: 4, Bias: 47.546267, T: 116, Avg. loss: 65.325488
Total training time: 0.00 seconds.
-- Epoch 1


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

-- Epoch 1
Norm: 5.86, NNZs: 4, Bias: 4.645582, T: 116, Avg. loss: 0.000622
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 8.95, NNZs: 4, Bias: 17.652629, T: 116, Avg. loss: 0.942246
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 46.54, NNZs: 4, Bias: -161.212554, T: 116, Avg. loss: 0.197038
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 5.86, NNZs: 4, Bias: 4.645906, T: 116, Avg. loss: 0.000622
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 8.61, NNZs: 4, Bias: 17.745127, T: 116, Avg. loss: 1.228567
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 46.65, NNZs: 4, Bias: -161.163468, T: 116, Avg. loss: 0.175086
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 5.86, NNZs: 4, Bias: 4.646065, T: 116, Avg. loss: 0.000622
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 8.67, NNZs: 4, Bias: 17.661657, T: 116, Avg. loss: 1.204323
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 46.66, NNZs: 4, Bias: -161.144094, T: 116, Avg. loss: 0.192339
Total training time: 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


-- Epoch 1
Norm: 5.86, NNZs: 4, Bias: 4.647228, T: 116, Avg. loss: 0.000622
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 8.77, NNZs: 4, Bias: 17.551165, T: 116, Avg. loss: 1.016677
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 46.76, NNZs: 4, Bias: -161.068836, T: 116, Avg. loss: 0.207327
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 5.86, NNZs: 4, Bias: 4.647451, T: 116, Avg. loss: 0.000622
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 8.58, NNZs: 4, Bias: 17.616096, T: 116, Avg. loss: 1.194040
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 46.83, NNZs: 4, Bias: -161.031327, T: 116, Avg. loss: 0.176386
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 5.86, NNZs: 4, Bias: 4.647759, T: 116, Avg. loss: 0.000622
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 8.49, NNZs: 4, Bias: 17.583147, T: 116, Avg. loss: 1.051203
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 46.55, NNZs: 4, Bias: -161.095240, T: 116, Avg. loss: 0.166571
Total training time: 

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished


## Check how well we did

During training we printed out the accuracy on the training dataset.

We use the test dataset to test how well the model performs to data that it hasn't seen before. This gives us an unbiased (hopefully) estimate of the true model performance for future data. This is what you would report as the final model accuracy.

In [25]:
print(f"The final model has an accuracy of {100*classifier_increment.score(X_test, y_test):.1f}%")

The final model has an accuracy of 100.0%


## Let's make some predictions of our test dataset

In [26]:
classifier_increment.predict(X_test)

dask.array<_predict, shape=(nan,), dtype=int64, chunksize=(nan,), chunktype=numpy.ndarray>

In [27]:
df_incremental = pd.DataFrame()

df_incremental["Truth"] = y_test.compute()  # Note the .compute()
df_incremental["Prediction"] =  classifier_increment.predict(X_test).compute()  # Note the .compute()

df_incremental

,Truth,Prediction
11,0,0
130,2,2
28,0,0
21,0,0
125,2,2
37,0,0
84,1,1
9,0,0
19,0,0
33,0,0


In [28]:
df_incremental["Truth"] = y_test.compute()  # Note the .compute()
df_incremental["Prediction"] =  classifier_increment.predict(X_test).compute()  # Note the .compute()


df_incremental["Truth"] = df_incremental["Truth"].map(map_values)
df_incremental["Prediction"] = df_incremental["Prediction"].map(map_values)

df_incremental["Match?"] = (df_incremental["Truth"] == df_incremental["Prediction"])

df_incremental

,Truth,Prediction,Match?
11,Iris Setosa,Iris Setosa,True
130,Iris Versicolour,Iris Versicolour,True
28,Iris Setosa,Iris Setosa,True
21,Iris Setosa,Iris Setosa,True
125,Iris Versicolour,Iris Versicolour,True
37,Iris Setosa,Iris Setosa,True
84,Iris Virginica,Iris Virginica,True
9,Iris Setosa,Iris Setosa,True
19,Iris Setosa,Iris Setosa,True
33,Iris Setosa,Iris Setosa,True


In [29]:
classifier_increment.coef_

array([[-4.65467476, -2.30950012,  0.22452259,  2.70853781],
       [ 1.49764027, -4.79324128, -0.04238401, -6.82797672],
       [33.31090183, 32.17124146, -5.39987549, -5.92758528]])

In [30]:
classifier_increment.scoring

'accuracy'

## We won't have time to go over the validation dataset but it's a good topic for another time

Take a look at [hyperparameter optimization](https://scikit-learn.org/stable/modules/grid_search.html) for the details.

In [31]:
classifier_increment.score(X_validate, y_validate)

1.0